# Setup 

## import libraries

In [ ]:
import numpy as np
import os 
import pandas as pd
import json
from langchain_community.llms import Ollama
from langchain import PromptTemplate
import re
import ast

## import data

In [ ]:
athenadata = open("Downloads/Athena_feedback.txt", encoding='windows-1252').read().splitlines()

In [ ]:
UserID = []
Message = []

for i in range(0,len(athenadata)):
    if i % 2 == 0:
        UserID.append(athenadata[i])
    elif i % 2 == 1:
        Message.append(athenadata[i])
    else:
        continue

In [ ]:
AUF_Df = pd.DataFrame({
    'UserID':UserID,
    'Message':Message
})

In [ ]:
AUF_Df.head()

## Select LLM

In [ ]:
# llm = Ollama(base_url='http://localhost:11434',
#              model="llama2")
# mistral_bot = Ollama(base_url='http://localhost:11434',
#              model="mistral")
ActiveLLM = Ollama(base_url='http://localhost:11434',
             model="llama3")


In [ ]:
def useLLM(input, template):
    
    output = ActiveLLM(
    template.format(
        userfeedback=input
        )
    )
    return output

## Initialize prompt

In [ ]:
UX_topic_prompt_SC = """
Role: You are in the role of a UX researcher evaluating user feedback to find what users like about our new software system. Those likes and dislikes will be eventually turned into design decisions.
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.

Instructions:
I will give you some text that is feedback from a user testing a new software system. 
This feedback may be long and cover multiple topics. In that case break down the response into the component topics and return only 1-4 main topics.
Also, summarize those main topics so that each topic is concicse, and make sure that none of the main topics are redundant. write those main topics and an explanation of those topics in a list format like so: ["main topic 1": "your explanation", "main topic 2": "your explanation", "main topic 3": "your explanation"].
Think through this process step by step.
Then repeat this process five times so that you have a 5 lists of main topics with explanations. 
Then review the topics you have written and choose the topics that are the most consistent and not redundant with the prvooius topics and add one final list of these best and most representative topics.

your final output should be a list of lists with 5 sets of main topics with explanation, and a final list of just the most consistent main topics with explanations for those topics.

Do not stop to ask me if you should process or do anything else except complete the task of creating 5 sets of potential main topcs and a final set that you think best catprues what you did the firts 5 times.

Here is an example:

Example: 
    User Feedback: "The search function was excellent, but the insight feature was poor"
    Output: [["search function: the users discuss the search function", "insight feature: the user said they didnt like the insights feature"],
            ["search: the user seems focused on the search abilities of the system", "insight: the user was focused on giving feedback about the insight system"],
            ["system": the user mentions a system", "insight: the user commented on the insight feature"],
            ["search: the user seems to discuss the excellency of search", "poor: the user said the system was poor"],
            ["search: the user seems focused on the search abilities of the system", "insight: the user was focused on giving feedback about the insight system"],
            ["search": the user focused on the excellency of the search, "insight: the user found the insights to be not useful"]]

User Feedback: {userfeedback}
Output: """

In [ ]:
prompt_templateT = PromptTemplate(
    input_variables=["userfeedback"],
    template=UX_topic_prompt_SC
)

# use actually process data

## get initial topics
notice that we processed just one message first because this is rather slow, 
so I recommend when you are adapting your prompt to try it on one message at a time first 
before running all of them as it take 30 minutes to an hour to run for everyone

In [ ]:
AUF_Dft1 = AUF_Df.iloc[0:1,]

In [ ]:
AUF_Dft1['Message'][0]

In [ ]:
AUF_Dft1['Topics'] = AUF_Dft1['Message'].apply(useLLM, template=prompt_templateT)

In [ ]:
AUF_Dft1['Topics'][0]

In [ ]:
AUF_Dft1['MainTopics'] = AUF_Dft1['Topics'].apply(lambda x: x.split('[')[-1])

In [ ]:
AUF_Dft1['MainTopics']

## do full df

In [ ]:
AUF_Df['Topics'] = AUF_Df['Message'].apply(useLLM, template=prompt_templateT)

In [ ]:
AUF_Df['Topics'].head()

In [ ]:
AUF_Df['Topics'][0]

In [ ]:
AUF_Df['MainTopics'] = AUF_Df['Topics'].apply(lambda x: x.split('[')[-1])

In [ ]:
AUF_Df['MainTopics'].head()

## use LLM to try to trim total number of topics down

In [ ]:
TopicSynthesizer_template = """
Role: You are in the role of a UX researcher working with other UX researchers evaluting user feedback to find what users like about our system. 
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.
We need to have a good understanding of what users like and dont like about our system so we can improve it.

Instructions:
Another UX researcher has already broken each of the user messages down into a few topics for each user. 
You will be given a list of sets of all of the topics that each user mentioned topics, and your job is to try to reduce all topics down to a few different major topics that reflect different parts of the system we should focus on. 
The set of major topics you generate should be about different parts of the system and not be redundant. You should try to provide only a few topics. 
For each topic you provide, also add an explanation of why you think that is a major theme and what you think we should do to address it.

Topics List: {topicslist}
Analysis: """

In [ ]:
prompt_templateMTY = PromptTemplate(
    input_variables=["topicslist"],
    template=TopicSynthesizer_template
)

In [ ]:
def useLLM_topicSynthesizer(input, template):
    
    output = ActiveLLM(
    template.format(
        topicslist=input
        )
    )
    return output

In [ ]:
#AUF_Df['MainTopics'].tolist()

In [ ]:
topics_list = AUF_Df['MainTopics'].tolist()

In [ ]:
Majortopics = useLLM_topicSynthesizer( topics_list ,template=prompt_templateMTY)

In [ ]:
Majortopics

# Additional Prompt templates

In [ ]:
UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. Some of these responses are long and cover mutliple topics.
Analyze the sentiment expressed in the user feedback and for each topic and provide classification of the overall sentiment of all the topics into 1 of 4 categories:
"positive sentiment", "negative sentiment", "neutral sentiment", or "mixed sentiment". 
If the text seems like it was not about the software then say simply say "NA". 
first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
Below are some examples to help you.

Example 1: 
    User Feedback: "The search function was excellent, but the insight feature was poor"
    Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed sentiment" , "mixed sentiment"]

Example 2:
    User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the functions did exactly what I expected."
    Output: ["The user expressed positive sentiment about all of the things they mentioned. this statement is postively sentiment" , "positive sentiment"]

Example 3:
    User Feedback: "The system was very difficult to use and not intuitive at all."
    Output: ["The user described the system as ndifficult and not intuitive. This statement is negatively sentiment." , "negative sentiment"]

Example 4:
    User Feedback: "I like ice cream."
    Analysis: ["The statement does not seem to be about the software system." , "NA"]

Example 5:
    User Feedback: "The software was fine and I finished my work."
    Analysis: ["The user did not express any strong sentiments about the system. This statement is of neutral sentiment." , "neutral sentiment"]

User Feedback: {userfeedback}
Analysis: """

In [ ]:
UX_topic_prompt = """
Role: You are in the role of a UX researcher evaluting user feedback to find what users like about our system. Those likes and displikes will be eventually turned into design decisions.
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.

Instructions:
I will give you some text that is feedback from a user testing a new software system. 
This feedback may be long and cover mutliple topics. In that case break down the response into the component topics and return only 1-4 main topics.
Also, summarize those main topics so that each one is only one or two words long, and make sure that none of the main topics are redundant.
Think through this process step by step.
Finally, Once you have determined a set of main topics, return those main topics in a list form like so: ["main topic 1", "main topic 2", "main topic 3"]. 
Do not return the users original feedback or any additional explanation.

Here are two examples:

Example1: 
    User Feedback: "The search function was excellent, but the insight feature was poor"
    Output: ["search function", "insight feature"]

Example 2:
    User Feedback: "The system was very difficult to use and not intuitive at all."
    Output: ["the system"]

User Feedback: {userfeedback}
Output: """

# Scraps

In [ ]:
AUF_Df['MainTopics'] = AUF_Df['Topics'].apply(lambda x: str.split(str.split(x, 'Final Output:')[-1].strip(' []\"').replace('"', '').replace(' ', ''),','))

In [ ]:
str.split(str.split(AUF_Df['Topics'][2], 'Final Output:')[-1].strip(' []\"').replace('"', '').replace(' ', ''),',')

In [ ]:
ast.literal_eval(AUF_Df['Topics'][2])

In [ ]:
prompt_template = PromptTemplate(
    template=UXSentiment_template
)

In [ ]:
AUF_Dft3 = AUF_Df.iloc[1:3,]

In [ ]:
AUF_Dft3['Topics'] = AUF_Dft3['Message'].apply(usemistral, template=prompt_templateT)

In [ ]:
AUF_Dft3['Topics']

In [ ]:
UF_Df['Analysis' ,'Valence'] = UF_Df['Message'].apply(usemistral, template=prompt_template)

In [ ]:
UF_Df

In [ ]:
UF_Df.iloc[2,2]

In [ ]:
UF_Df.iloc[2,4]

In [ ]:
# UF_Df['Analysis']

In [ ]:
AUF_Dft = AUF_Df.iloc[:1,]

In [ ]:
AUF_Dft['Analysis' ,'Valence'] = AUF_Dft['Message'].apply(usemistral, template=prompt_template)

In [ ]:
AUF_Dft.iloc[0,1]

In [ ]:
AUF_Dft.iloc[0,2]

In [ ]:
AUF_Dft2 = AUF_Df.iloc[:2,]

In [ ]:
AUF_Dft2['Topics'] = AUF_Dft['Message'].apply(usemistral, template=prompt_templateT)

In [ ]:
AUF_Dft2['Message'][0]

In [ ]:
AUF_Dft2['Topics'][0]

In [ ]:
AUF_Dft2['Message'][1]

In [ ]:
AUF_Dft2['Topics'][1]

In [ ]:
UF_Df['Valence'][2]

In [ ]:
UF_Df['Message'][0]

In [ ]:
UF_Df['Message'][1]

In [ ]:
UF_Df['Message'][3]

In [ ]:
mistral_bot(
    prompt_template.format(
        userfeedback=
    )
)

In [ ]:
print(llm('Hello, how are you?'))

In [ ]:
teststore = llm("say testing")

In [ ]:
mistral_bot = Ollama(base_url='http://localhost:11434',
             model="mistral")

In [ ]:
print(mistral_bot("say I am mistral"))

In [ ]:
print(mistral_bot({
    "role": "user",
    "content": "why is the sky blue?"
}))

In [ ]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: {previous_context}

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query","previous_context"],
    template=template
)

In [ ]:
print(
    prompt_template.format(
        query="Who was the first president of the united states?",
        previous_context=""
    )
)

In [ ]:
chat_history = ""

In [ ]:
myquery = "Who was the first president of the united states?"
current_response = mistral_bot(
    prompt_template.format(
        query=myquery,
        previous_context = chat_history
        
    )
)
chat_history += "User Question:" + myquery + "\n"
chat_history += "LLM Response:" + current_response + "\n"
print(current_response)

In [ ]:
print(
    prompt_template.format(
        query="Did he write lord of the rings?",
        previous_context=chat_history
    )
)

In [ ]:
myquery = "What dog was in the sandlot movie?"

In [ ]:
print(
    prompt_template.format(
        query="myquery",
        previous_context=chat_history
    )
)

In [ ]:
if len(chat_history) > 3000:
    print("youre are running out of tokens, just a heads up.")
current_response = mistral_bot(
    prompt_template.format(
        query=myquery,
        previous_context = chat_history
        
    )
)
chat_history += myquery
chat_history += current_response
print(current_response)

In [ ]:
# myquery = input()
# if len(chat_history) > 3000:
#     print("youre are running out of tokens, just a heads up.")
# current_response = mistral_bot(
#     prompt_template.format(
#         query=myquery,
#         previous_context = chat_history
        
#     )
# )
# chat_history += myquery
# chat_history += current_response
# print(current_response)

In [ ]:
# OLd User sent template
#UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. 
# Analyze the emotional valence expressed in the user feedback and then classify the user feedback based on the emotional valence of the text into 1 of 4 categories:
# "positive valence", "negative valence", "neutral valence", or of "mixed valence". 
# If the text seems like it was not about the software then say simply say "NA". 
# first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
# Below are some examples to help you.

# Example 1: 
#     User Feedback: "The search function was excellent, but the insight feature was poor"
#     Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed valence", "mixed valence"]
#     Analysis: "mixed valence"

# Example 2:
#     User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the fucntions did exactly what I expected."
#     Analysis: "positive valence"

# Example 3:
#     User Feedback: "The system was very difficult to use and not intuitive at all."
#     Analysis: "negative valence"

# Example 4:
#     User Feedback: "I like cookies."
#     Analysis: "NA"

# Example 5:
#     User Feedback: "The software was fine and I finished my work."
#     Analysis: "neutral valence"

# User Feedback: {userfeedback}
# Analysis: """

In [ ]:
# UserID = []
# System = []
# Message = []

# for i in range(0,len(filelines)):
#     if filelines[i] == '':
#         continue
#     elif filelines[i][-8:] == 'GMT 2023' :
#         UserID.append(int(filelines[i].split(' ')[0].split('-')[1]))
#         System.append(filelines[i].split(' ')[2][0])
#     else:
#         Message.append(filelines[i])
        
                      
        

In [ ]:
#ux sentiment template 2
# UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. 
# Analyze the emotional valence expressed in the user feedback and then classify the user feedback based on the emotional valence of the text into 1 of 4 categories:
# "positive valence", "negative valence", "neutral valence", or "mixed valence". 
# If the text seems like it was not about the software then say simply say "NA". 
# first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
# Below are some examples to help you.

# Example 1: 
#     User Feedback: "The search function was excellent, but the insight feature was poor"
#     Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed valence" , "mixed valence"]

# Example 2:
#     User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the fucntions did exactly what I expected."
#     Output: ["The user expressed positive sentiment about all of the things they mentioned. this statement is postively valenced" , "positive valence"]

# Example 3:
#     User Feedback: "The system was very difficult to use and not intuitive at all."
#     Output: ["The user described the system as ndifficult and not intuitive. This statement is negatively valenced." , "negative valence"]

# Example 4:
#     User Feedback: "I like ice cream."
#     Analysis: ["The statement does not seem to be about the software system." , "NA"]

# Example 5:
#     User Feedback: "The software was fine and I finished my work."
#     Analysis: ["The user did not express any strong sentiments about the system. This statement is of neutral valence." , "neutral valence"]

# User Feedback: {userfeedback}
# Analysis: """

In [ ]:
UF_Df

In [ ]:
UserID

In [ ]:
for i in range(0,15):
    if i % 3 == 0:
        print(i)

In [ ]:
UserID

In [ ]:
filelines[0].split(' ')[0].split('-')[1]

In [ ]:
filelines[0].split(' ')[2][0]

In [ ]:
os.getcwd()
# dir_path = os.path.dirname(os.path.realpath(__file__))


In [ ]:
dir_path = os.path.dirname(os.path.realpath("FAKE_AthenUserFeedback.txt"))

In [ ]:
json.load("Downloads/FAKE_AthenUserFeedback.txt")

In [ ]:
# Opening JSON file
f = open('Downloads/FAKE_AthenUserFeedback.txt')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data['emp_details']:
    print(i)
 
# Closing file
f.close()

In [ ]:

# FormLLM_Template = """

# Instructions: {}

# Context: {}

# User Input: {}

# Output: """

In [ ]:

# TwoKilo_Template = """

# Instructions: {
# You are a form checker, your job is to ...
# Here are the instructions for how to fill out a 2 Kilo form. 
# Here are detail about how the information in the for is used.

# Here are some examples of good an bad responses to certain fields of the 2 kilo form

# Example1

# Example 2

# Example 3
# }

# Context: {Here is information about the role that the current sailor serves,
# Here is infromatoin about the ship and systems that they work on, and 
# Here is infromatino about the jargon or slang associated with that role
# Here is the infromation the sailor has already put into the form}

# User Input: {The field the user is currently filling is ...

# here is the users input to the field
# }

# Output: """